In [2]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [14]:
df = pd.read_csv("../Data/Interim/some_cmd_split.csv", sep=",")
df = df.loc[:,["full_text", "text1", "text2", "text3", "subset"]]
print(df.shape)
# добавляем кома
df_onecmd = pd.read_csv("../Data/Interim/24052022_onecmd_split.csv", sep=",")
df_onecmd = df_onecmd.sample(5000, random_state=42)
df_onecmd = df_onecmd.loc[df_onecmd["subset"].isin(["train", "valid"]), ["x", "subset"]]
df_onecmd["full_text"] = df_onecmd["x"]
df_onecmd["text1"] = df_onecmd["x"]
df_onecmd["text2"] = np.empty_like(df_onecmd["x"], dtype=np.float32)
df_onecmd["text3"] = np.empty_like(df_onecmd["x"], dtype=np.float32)

df = pd.concat([df, df_onecmd])
df.index= range(len(df))

(16722, 5)


/tmp/ipykernel_5886/4095646851.py:1: DtypeWarning: Columns (3,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Data/Interim/some_cmd_split.csv", sep=",")


In [15]:
nerInput_train = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
nerInput_valid = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
nerInput = None
for ir, row in df.iterrows():
    if row["subset"]=="train":
        nerInput = nerInput_train
    if row["subset"]=="valid":
        nerInput = nerInput_valid

    start_1 = row["full_text"].find(row["text1"])
    assert start_1 >= 0
    start_2 = None
    if type(row['text2']) != float:
        start_2 = row["full_text"].find(row["text2"], start_1 + len(row["text1"]))    
        assert start_2 >= start_1
    start_3 = None
    if type(row['text3']) != float:
        start_3 = row["full_text"].find(row["text3"], start_2 + len(row["text2"]))
        assert start_3 >= start_2
    lastWordPos = 0
    for word in row["full_text"].split():
        nerInput["sentence_id"].append(ir)
        nerInput["words"].append(word)
        wordPos = row["full_text"].find(word, lastWordPos)
        assert wordPos!=-1
        lastWordPos = wordPos + len(word)
        if lastWordPos == (start_1+len(row["text1"].strip())):
            nerInput["labels"].append("[SEP]")
        elif (start_2 is not None) and lastWordPos == (start_2+len(row["text2"].strip())):
            nerInput["labels"].append("[SEP]")
        elif (start_3 is not None) and lastWordPos == (start_3+len(row["text3"].strip())):
            nerInput["labels"].append("[SEP]")
        else:
            nerInput["labels"].append("O")
train_df = pd.DataFrame(nerInput_train)
valid_df = pd.DataFrame(nerInput_valid)

In [5]:
for ir, row in df.iterrows():
    cmd_count = 1
    if type(row["text2"])==str:
        cmd_count+=1
    if type(row["text3"])==str:
        cmd_count+=1
    if ir in train_df["sentence_id"].values:
        assert train_df[train_df["sentence_id"]==ir]["labels"].value_counts()["[SEP]"]==cmd_count
    else:
        assert valid_df[valid_df["sentence_id"]==ir]["labels"].value_counts()["[SEP]"]==cmd_count

In [6]:
ir = 5000
print(df.loc[ir, "full_text"])
ner_commands= [[],[],[]]
current_command = 0
for ir_2, row in train_df[train_df["sentence_id"]==ir].iterrows():
    print(row["words"], row["labels"])
    ner_commands[current_command].append(row["words"])
    if row["labels"]=="[SEP]":
        current_command += 1
print(df.loc[ir, "text1"], "|", " ".join(ner_commands[0]))
print(df.loc[ir, "text2"], "|", " ".join(ner_commands[1]))
print(df.loc[ir, "text3"], "|", " ".join(ner_commands[2]))

обойди окрестности по кругу размером 17 метров после всего тяни на южный край на 4 м а там за лесом располагается камушек который нужно обскакивать
обойди O
окрестности O
по O
кругу O
размером O
17 O
метров [SEP]
после O
всего O
тяни O
на O
южный O
край O
на O
4 O
м [SEP]
а O
там O
за O
лесом O
располагается O
камушек O
который O
нужно O
обскакивать [SEP]
обойди окрестности по кругу размером 17 метров | обойди окрестности по кругу размером 17 метров
тяни на южный край на 4 м | после всего тяни на южный край на 4 м
за лесом располагается камушек который нужно обскакивать | а там за лесом располагается камушек который нужно обскакивать


In [12]:
from simpletransformers.ner import NERModel, NERArgs
model_args = NERArgs()
model_args.evaluate_during_training = True
model_args.num_train_epochs = 10
model_args.output_dir = "/s/ls4/users/grartem/RL_robots/CommandClassifier/models/TokenCLS_sepPredictor_ipynb"

model = NERModel(
    "bert", 
    #"/s/ls4/users/grartem/HuggingFace/models/rubert-tiny2",
    "/s/ls4/users/grartem/RL_robots/CommandClassifier/models/TokenCLS_sepPredictor_ipynb/checkpoint-20664-epoch-9/",
    args=model_args,
    labels=["O", "[SEP]"]
)

In [8]:
# Train the model
model.train_model(train_df, eval_data=valid_df)

  0%|          | 0/29 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/2296 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [SEP] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


  0%|          | 0/7 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/385 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [SEP] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


(22960,
 defaultdict(list,
             {'global_step': [2000,
               2296,
               4000,
               4592,
               6000,
               6888,
               8000,
               9184,
               10000,
               11480,
               12000,
               13776,
               14000,
               16000,
               16072,
               18000,
               18368,
               20000,
               20664,
               22000,
               22960],
              'train_loss': [0.0008940696716308594,
               0.00045271823182702065,
               0.0003213851014152169,
               8.192089444492012e-05,
               4.287319825380109e-05,
               0.00010234460205538198,
               3.014238245668821e-05,
               2.8947124519618228e-05,
               2.541490175644867e-05,
               1.9548968339222483e-05,
               9.970437531592324e-06,
               9.085940291697625e-06,
               1.064881871570

In [17]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(valid_df)

  0%|          | 0/7 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/385 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [SEP] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [16]:
predictions, raw_outputs = model.predict(df[df["subset"]=="valid"]["full_text"])

  0%|          | 0/7 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/385 [00:00<?, ?it/s]

# оценка

In [17]:
from sklearn.metrics import classification_report
flat_token_predictions = [list(x.values())[0] for sent in predictions for x in sent]
flat_token_true = valid_df["labels"].values
print(classification_report(flat_token_true, flat_token_predictions))

              precision    recall  f1-score   support

           O       1.00      1.00      1.00     26655
       [SEP]       1.00      1.00      1.00      5602

    accuracy                           1.00     32257
   macro avg       1.00      1.00      1.00     32257
weighted avg       1.00      1.00      1.00     32257



In [24]:
i, errors = 0, 0
for ir, row in df[df["subset"]=="valid"].iterrows():
    text1, text2, text3 = [], [], []
    cmd_i = 0
    for token in predictions[i]:
        token, cls = list(token.items())[0]
        if cmd_i==0:
            text1.append(token)
        elif cmd_i==1:
            text2.append(token)
        elif cmd_i==2:
            text3.append(token)
        if cls=="[SEP]":
            cmd_i += 1
    text1 = " ".join(text1)
    text2 = " ".join(text2)
    text3 = " ".join(text3)
    
    try:
        assert row["text1"].strip() in text1
        assert (text2=="" and type(row["text2"]) is float) or row["text2"].strip() in text2
        assert (text3=="" and type(row["text3"]) is float) or row["text3"].strip() in text3
        #assert (text3=="" and np.isnan(row["text3"])) or text3 == row["text3"].strip()
    except AssertionError:
        print("full_text:", row["full_text"])
        print("text1:", row["text1"], "predict:", text1)
        print("text2:", row["text2"], "predict:", text2)
        print("text3:", row["text3"], "predict:", text3)
        print()
        errors += 1
    i+=1
print("Errors", errors)

full_text: убеги к мужчине слева  неподалеку от валуна располагается упавшее дерево которое нужно разглядывать
text1: убеги к мужчине слева  predict: убеги к мужчине
text2: неподалеку от валуна располагается упавшее дерево которое нужно разглядывать predict: слева
text3: nan predict: неподалеку от валуна располагается упавшее дерево которое нужно разглядывать

full_text: за лампой находится дубрава которую приказывается отыскать перед мужчиной дуб который приказывается рассматривать дальше пытайся шлёпать на 9m
text1: за лампой находится дубрава которую приказывается отыскать predict: за лампой находится дубрава которую приказывается отыскать перед мужчиной дуб который приказывается рассматривать
text2: перед мужчиной дуб который приказывается рассматривать predict: дальше пытайся шлёпать на 9m
text3: пытайся шлёпать на 9m predict: 

full_text: выходи к разрыву слева от дуба что находится правее пригорка в дальнейшем отходи в сторону востока семени в сторону запада 1м
text1: выходи к р

# проверим, как она хорошо предсказывает фразы из 1 команды, если не видела их

In [19]:
df = pd.read_csv("../Data/Interim/24052022_onecmd_split.csv", sep=",")
predictions, raw_outputs = model.predict(df["x"])

  0%|          | 0/28 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/2688 [00:00<?, ?it/s]

In [9]:
errors_count = 0
for phrase in predictions:
    classes = [list(x.values())[0] for x in phrase]
    if classes[-1]=="[SEP]" and len([x for x in classes if x=="[SEP]"])==1:
        continue
    else:
        errors_count+=1
errors_count, len(predictions)

(270, 21504)

In [20]:
errors_count = 0
for phrase in predictions:
    classes = [list(x.values())[0] for x in phrase]
    if classes[-1]=="[SEP]" and len([x for x in classes if x=="[SEP]"])==1:
        continue
    else:
        errors_count+=1
errors_count, len(predictions)

(246, 21504)